In [2]:
import sqlite3
import pandas as pd

In [3]:
con = sqlite3.connect(":memory:")
cur = con.cursor()

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [4]:
link = 'https://raw.githubusercontent.com/eric-bunch/boston_housing/master/boston.csv'

In [5]:
data = pd.read_csv(link)
data.rename(columns = {'MDEV':'MEDV'}, inplace=True)

In [6]:
data.to_sql("boston", con, index=False, if_exists="replace")

In [7]:
pd.read_sql(
    """
    SELECT * FROM boston
    limit 5
    """,
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


1. Выведите количество пустых значений по каждой из колонок (название колонки, кол-во пустых значений)

In [8]:
df = pd.read_sql(
    """
    SELECT * FROM boston
    limit 5
    """,
    con,
)

for col in df.columns:
    qr = ("""
        SELECT COUNT() AS CNT FROM boston WHERE {col} IS NULL
        """).format(col = col)
    print(qr)
    tmp = pd.read_sql(qr,con)['CNT'][0]
    print(col + ' пустых столбцов: ' + str(tmp))

for col in df.columns:
    qr = ("""
        SELECT COUNT() AS CNT FROM boston WHERE {col} = 0
        """).format(col = col)
    print(qr)
    tmp = pd.read_sql(qr,con)['CNT'][0]
    print(col + ' нулевых столбцов: ' + str(tmp))


        SELECT COUNT() AS CNT FROM boston WHERE CRIM IS NULL
        
CRIM пустых столбцов: 0

        SELECT COUNT() AS CNT FROM boston WHERE ZN IS NULL
        
ZN пустых столбцов: 0

        SELECT COUNT() AS CNT FROM boston WHERE INDUS IS NULL
        
INDUS пустых столбцов: 0

        SELECT COUNT() AS CNT FROM boston WHERE CHAS IS NULL
        
CHAS пустых столбцов: 0

        SELECT COUNT() AS CNT FROM boston WHERE NOX IS NULL
        
NOX пустых столбцов: 0

        SELECT COUNT() AS CNT FROM boston WHERE RM IS NULL
        
RM пустых столбцов: 0

        SELECT COUNT() AS CNT FROM boston WHERE AGE IS NULL
        
AGE пустых столбцов: 0

        SELECT COUNT() AS CNT FROM boston WHERE DIS IS NULL
        
DIS пустых столбцов: 0

        SELECT COUNT() AS CNT FROM boston WHERE RAD IS NULL
        
RAD пустых столбцов: 0

        SELECT COUNT() AS CNT FROM boston WHERE TAX IS NULL
        
TAX пустых столбцов: 0

        SELECT COUNT() AS CNT FROM boston WHERE PTRATIO IS NULL
 

2. Выведите количество уникальных значений по каждой из колонок (название колонки, кол-во уникальных значений)

In [9]:
df = pd.read_sql(
    """
    SELECT * FROM boston
    limit 5
    """,
    con,
)

for col in df.columns:
    qr = ("""
        SELECT COUNT(DISTINCT {col}) AS CNT FROM boston
        """).format(col = col)
    print(qr)
    tmp = pd.read_sql(qr,con)['CNT'][0]
    print(col + ' уникальных значений: ' + str(tmp))


        SELECT COUNT(DISTINCT CRIM) AS CNT FROM boston
        
CRIM уникальных значений: 504

        SELECT COUNT(DISTINCT ZN) AS CNT FROM boston
        
ZN уникальных значений: 26

        SELECT COUNT(DISTINCT INDUS) AS CNT FROM boston
        
INDUS уникальных значений: 76

        SELECT COUNT(DISTINCT CHAS) AS CNT FROM boston
        
CHAS уникальных значений: 2

        SELECT COUNT(DISTINCT NOX) AS CNT FROM boston
        
NOX уникальных значений: 81

        SELECT COUNT(DISTINCT RM) AS CNT FROM boston
        
RM уникальных значений: 446

        SELECT COUNT(DISTINCT AGE) AS CNT FROM boston
        
AGE уникальных значений: 356

        SELECT COUNT(DISTINCT DIS) AS CNT FROM boston
        
DIS уникальных значений: 412

        SELECT COUNT(DISTINCT RAD) AS CNT FROM boston
        
RAD уникальных значений: 9

        SELECT COUNT(DISTINCT TAX) AS CNT FROM boston
        
TAX уникальных значений: 66

        SELECT COUNT(DISTINCT PTRATIO) AS CNT FROM boston
        
PTRATI

3. Выведите колонки, у которых медиана равна минимальному значению (название колонки).
Напишите какой вывод можно сделать по данным в этих колонках

In [10]:
df = pd.read_sql(
    """
    SELECT * FROM boston
    limit 5
    """,
    con,
)

for col in df.columns:
    mediana_qr = """
        SELECT AVG({col}) as mediana
        FROM (SELECT {col}
              FROM boston
              ORDER BY {col}
              LIMIT 2
              OFFSET (SELECT (COUNT(*) - 1) / 2
                      FROM boston))
    """.format(col = col)
    mediana = pd.read_sql(mediana_qr,con)['mediana'][0]
    qr = ("""
        SELECT ({mediana} - MIN({col})) as RES FROM boston
        """).format(col = col, mediana = mediana)
    #print(qr)
    if (pd.read_sql(qr,con)['RES'][0] == 0):
        print(col + ' по этому столбцу больше половины значений равны минимальному')

ZN по этому столбцу больше половины значений равны минимальному
CHAS по этому столбцу больше половины значений равны минимальному


4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. 
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [11]:
cnt_list = [25,50,100,200,300]
res = pd.DataFrame()

for cnt in cnt_list:
    res_tmp = pd.read_sql(
    """
    SELECT {cnt} AS CNT, MAX AS AVG_RM_MAX, MIN AS ABG_RM_MIN , b.MAX - a.MIN as RES FROM
    (SELECT AVG(RM) AS MIN
    FROM (SELECT * FROM boston
    ORDER BY MEDV
    limit {cnt})) a
    CROSS JOIN 
    (SELECT AVG(RM) AS MAX
    FROM boston WHERE MEDV = (select MAX(MEDV) from boston)) b    
    """.format(cnt = str(cnt)),
    con)
    res = pd.concat([res,res_tmp], ignore_index = True)
print('Кол-во комнат достаточно сильно влияет на стоимость')
res

Кол-во комнат достаточно сильно влияет на стоимость


,CNT,AVG_RM_MAX,ABG_RM_MIN,RES
0,25,7.484,5.747840,1.736160
1,50,7.484,5.753240,1.730760
2,100,7.484,5.887120,1.596880
3,200,7.484,5.911705,1.572295
4,300,7.484,5.972227,1.511773


5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг). Напишите какой вывод можно сделать по этим данным.

In [12]:
pd.read_sql(
    """
    SELECT LSTAT, MEDV, rank() OVER (PARTITION BY MEDV ORDER BY LSTAT DESC) rank_by_lstat
    FROM boston
    WHERE MEDV = (SELECT MAX(MEDV) FROM boston)
    """,
    con,
)

,LSTAT,MEDV,rank_by_lstat
0,9.53,50.0,1
1,8.88,50.0,2
2,7.44,50.0,3
3,5.12,50.0,4
4,4.63,50.0,5
5,4.45,50.0,6
6,3.73,50.0,7
7,3.70,50.0,8
8,3.32,50.0,9
9,3.26,50.0,10


6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [13]:
pd.read_sql(
    """
    SELECT DISTINCT CHAS, AVG(MEDV) OVER (PARTITION BY CHAS) as MEAN_MEDV FROM boston
    """,
    con,
)

,CHAS,MEAN_MEDV
0,0.0,22.093843
1,1.0,28.440000


7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки). Напишите какой вывод можно сделать по этим данным.

In [14]:
res = pd.DataFrame()
df = pd.read_sql(
    """
    SELECT * FROM boston
    limit 5
    """,
    con,
)

for col in df.columns:
    qr = """
            SELECT *,'{col}' AS COL FROM
            (SELECT AVG({col}) AS AVG_NO_CHAS
            FROM (SELECT * FROM boston
            WHERE CHAS = 0)) a
            CROSS JOIN 
            (SELECT AVG({col}) AS AVG_YES_CHAS
            FROM (SELECT * FROM boston
            WHERE CHAS = 1)) b 
            WHERE AVG_YES_CHAS > AVG_NO_CHAS
    """.format(col = col)
    res_tmp = pd.read_sql(qr,con)
    res = pd.concat([res,res_tmp], ignore_index = True)

print('Дома у реки старше, в них больше комнат, они дороже, в них больше акров неторгового бизнеса, больше оксидов азота и больше чернокожих')
res

Дома у реки старше, в них больше комнат, они дороже, в них больше акров неторгового бизнеса, больше оксидов азота и больше чернокожих


,AVG_NO_CHAS,AVG_YES_CHAS,COL
0,11.019193,12.719143,INDUS
1,0.000000,1.000000,CHAS
2,0.551817,0.593426,NOX
3,6.267174,6.519600,RM
4,67.911677,77.500000,AGE
5,355.461040,372.997429,B
6,22.093843,28.440000,MEDV


8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [15]:
res = pd.read_sql(
    """
        WITH indus AS (SELECT INDUS, NTILE(10) OVER (ORDER BY INDUS) AS percentile
                   FROM boston),
                   
        nox AS (SELECT NOX, NTILE(10) OVER (ORDER BY NOX) AS percentile
                   FROM boston)
                   
        select a.percentile * 10 as percentile ,a.indus,b.nox from (SELECT percentile, MAX(indus) as indus
            FROM indus
            GROUP BY percentile) a
        LEFT JOIN (SELECT percentile, MAX(nox) as nox
            FROM nox
            GROUP BY percentile) b
        ON a.percentile = b.percentile      
        ;   
    """,
    con)
print('Связь между долей промышленной застройки и концентрацией оксидов азота прослеживается')
res

Связь между долей промышленной застройки и концентрацией оксидов азота прослеживается


,percentile,indus,nox
0,10,2.89,0.426
1,20,4.39,0.442
2,30,5.96,0.472
3,40,7.38,0.507
4,50,9.69,0.538
5,60,13.89,0.580
6,70,18.10,0.609
7,80,18.10,0.671
8,90,19.58,0.713
9,100,27.74,0.871
